# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
import sys
sys.path.append("..")
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Import csv file and read

csv = os.path.join('..','output_data', 'cities.csv')

city_df = pd.read_csv(csv)
city_df


,City,Lat,Lng,Max Temp,Humidity,Clouds,Wind Speed,Country,Date
0,Buchanan,5.8808,-10.0467,32.00,70,75,5.14,LR,1619101829
1,Nísia Floresta,-6.0911,-35.2086,30.00,70,75,5.14,BR,1619101830
2,Tumut,-35.3000,148.2167,10.00,81,90,1.33,AU,1619101830
3,Rikitea,-23.1203,-134.9692,25.57,73,99,8.03,PF,1619101830
4,Jamestown,42.0970,-79.2353,0.56,69,90,6.17,US,1619101830
...,...,...,...,...,...,...,...,...,...
453,Yuxia,34.0615,108.6291,16.11,94,88,3.00,CN,1619101903
454,Rockhampton,-23.3833,150.5000,20.00,94,40,0.74,AU,1619101903
455,Morwa,22.9000,73.8333,33.40,17,2,2.26,IN,1619101904
456,Saint Paul Harbor,57.7900,-152.4072,5.00,93,90,6.17,US,1619101904


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [4]:
# Create a Heatmap layer
locations = city_df[["Lat", "Lng"]]
humidity = city_df["Humidity"]

figure = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False)
heat_layer.max_intensity = 100
heat_layer.point_radius = 3

figure.add_layer(heat_layer)
figure

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [45]:
# Narrow down the DataFrame to find your ideal weather condition and use dropna to remove null values

narrowed_df = city_df.copy()
narrowed_df = narrowed_df[narrowed_df['Max Temp'] >= 20]
narrowed_df = narrowed_df[narrowed_df['Max Temp'] <= 30]  
narrowed_df = narrowed_df[narrowed_df['Wind Speed'] <= 10]
narrowed_df = narrowed_df[narrowed_df['Clouds'] == 0]
narrowed_df = narrowed_df.dropna()
narrowed_df = narrowed_df.reset_index()
narrowed_df


,index,City,Lat,Lng,Max Temp,Humidity,Clouds,Wind Speed,Country,Date
0,8,Hong Kong,22.2855,114.1577,25.56,77,0,4.02,HK,1619101597
1,35,Haimen,23.1935,116.6142,24.00,73,0,4.00,CN,1619101836
2,54,Saint-Philippe,-21.3585,55.7679,26.00,78,0,7.72,RE,1619101839
3,57,Cape Town,-33.9258,18.4232,23.89,53,0,3.09,ZA,1619101819
4,79,Port Elizabeth,-33.9180,25.5701,21.00,78,0,7.72,ZA,1619101808
5,148,Salalah,17.0151,54.0924,30.00,66,0,4.63,OM,1619101855
6,160,Palmeiras de Goiás,-16.8050,-49.9258,27.23,40,0,1.41,BR,1619101856
7,260,Ciocăneşti,44.6000,25.8500,20.56,32,0,5.66,RO,1619101872
8,293,Arrondissement de Libourne,44.9167,-0.2500,22.78,33,0,5.66,FR,1619101878
9,329,Navoiy,40.0844,65.3792,20.00,22,0,8.23,UZ,1619101883


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [46]:
# Create hotel dataframe variable
hotel_df = narrowed_df

In [47]:
# Add a new column to dataframe "Hotel Name"
hotel_df['Hotel Name'] = ""
hotel_df

,index,City,Lat,Lng,Max Temp,Humidity,Clouds,Wind Speed,Country,Date,Hotel Name
0,8,Hong Kong,22.2855,114.1577,25.56,77,0,4.02,HK,1619101597,
1,35,Haimen,23.1935,116.6142,24.00,73,0,4.00,CN,1619101836,
2,54,Saint-Philippe,-21.3585,55.7679,26.00,78,0,7.72,RE,1619101839,
3,57,Cape Town,-33.9258,18.4232,23.89,53,0,3.09,ZA,1619101819,
4,79,Port Elizabeth,-33.9180,25.5701,21.00,78,0,7.72,ZA,1619101808,
5,148,Salalah,17.0151,54.0924,30.00,66,0,4.63,OM,1619101855,
6,160,Palmeiras de Goiás,-16.8050,-49.9258,27.23,40,0,1.41,BR,1619101856,
7,260,Ciocăneşti,44.6000,25.8500,20.56,32,0,5.66,RO,1619101872,
8,293,Arrondissement de Libourne,44.9167,-0.2500,22.78,33,0,5.66,FR,1619101878,
9,329,Navoiy,40.0844,65.3792,20.00,22,0,8.23,UZ,1619101883,


In [48]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [51]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
figure.add_layer(markers)

# Display figure
figure

Figure(layout=FigureLayout(height='420px'))